## NOT MY CODE

In [ ]:
import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from numpy.random import uniform
import pdb

In [ ]:
def find_color_values_using_trackbar(frame):

    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
 
    def nothing(x):
        pass

    cv.namedWindow("Trackbar") 
    cv.createTrackbar("LH", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LS", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("LV", "Trackbar", 0, 255, nothing)
    cv.createTrackbar("UH", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("US", "Trackbar", 255, 255, nothing)
    cv.createTrackbar("UV", "Trackbar", 255, 255, nothing)
    
    
    while True:

        l_h = cv.getTrackbarPos("LH", "Trackbar")
        l_s = cv.getTrackbarPos("LS", "Trackbar")
        l_v = cv.getTrackbarPos("LV", "Trackbar")
        u_h = cv.getTrackbarPos("UH", "Trackbar")
        u_s = cv.getTrackbarPos("US", "Trackbar")
        u_v = cv.getTrackbarPos("UV", "Trackbar")


        l = np.array([l_h, l_s, l_v])
        u = np.array([u_h, u_s, u_v])
        mask_table_hsv = cv.inRange(frame_hsv, l, u)        

        res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)    
        cv.imshow("Frame", frame)
        cv.imshow("Mask", mask_table_hsv)
        cv.imshow("Res", res)

        if cv.waitKey(25) & 0xFF == ord('q'):
                break
    cv.destroyAllWindows()

In [3]:
img = cv.imread("fullNewTrack.jpg")
# img=cv.resize(img,(0,0),fx=0.20,fy=0.20)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
find_color_values_using_trackbar(img)

In [4]:
# Python program to illustrate 
# Otsu thresholding type on an image 

# organizing imports q
 
# path to input image is specified and 
# image is loaded with imread command 
image1 = cv.imread('fullNewTrack.jpg') 

# cv2.cvtColor is applied over the 
# image input with applied parameters 
# to convert the image in grayscale 
img = cv.cvtColor(image1, cv.COLOR_BGR2GRAY) 

# applying Otsu thresholding 
# as an extra flag in binary 
# thresholding
blur = cv.GaussianBlur(img,(5,5),0)
ret, thresh1 = cv.threshold(blur,205,255,cv.THRESH_BINARY)

# the window showing output image		 
# with the corresponding thresholding		 
# techniques applied to the input image	 
cv.imshow('Otsu Threshold', thresh1)		 
	
# De-allocate any associated memory usage		 
cv.waitKey(0)
cv.destroyAllWindows()	 


In [5]:
low_yellow = (75,0,144)
high_yellow = (250,40,255)
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
# kernel = np.ones((6,6),np.uint8)
mask_yellow_hsv = cv.bitwise_not(mask_yellow_hsv)
img1_bg = cv.bitwise_and(img_hsv,img_hsv,mask = mask_yellow_hsv)
# mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_CLOSE, kernel)
# cv.imshow('img_initial', img1_bg)
cv.imshow('mask_yellow_hsv', mask_yellow_hsv)
cv.waitKey(0)
cv.destroyAllWindows()

error: OpenCV(4.10.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x9faa3a91::Set<3,-1,-1>,struct cv::impl::A0x9faa3a91::Set<0,5,-1>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [ ]:
a = [2,1]
a = a.sort()

In [ ]:
def make_coordinates(image, line_parameters,Y1,Y2):
    slope, intercept = line_parameters
    y1 = int(Y1)
    y2 = int(Y2)
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

def compare_func(element):
    return element[0][1]

def average_slope_intercept(image, lines,parts):
    height, _,_ = image.shape
    final_lines = []
    # i = 0
    for i in range(parts):
        left_fit = []
        right_fit = []
        sorted_lines = sorted(lines,key=compare_func)
        # print(sorted_lines)
        # sorted_lines = lines.sorted()
        print((i+1)*1/parts*height)
        for line in sorted_lines:
            x1, y1, x2, y2 = line.reshape(4)
            if i*1/parts*height < y2 < (i+1)*1/parts*height:
                # print(y1)
                parameters = np.polyfit((x1, x2), (y1, y2), 1)
                slope = parameters[0]
                intercept = parameters[1]
                # print(slope)
                if slope < 0:
                    left_fit.append((slope, intercept))
                else:
                    right_fit.append((slope, intercept))
        
        # print(left_fit)
        if left_fit:
            left_fit_average = np.average(left_fit, axis=0)
            left_line = make_coordinates(image, left_fit_average,(i+1)*1/parts*height,i*1/parts*height)
            final_lines.append(left_line)
        if right_fit:
            right_fit_average = np.average(right_fit, axis=0)
            right_line = make_coordinates(image, right_fit_average,(i+1)*1/parts*height,i*1/parts*height)
            final_lines.append(right_line)
    return np.array(final_lines)

def canny(image):
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray,(5, 5), 0)
    canny = cv.Canny(blur, 50, 150)
    return canny

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            cv.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 1)
    return line_image

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
    [(200, height), (1100, height), (550, 250)]
    ])
    mask = np.zeros_like(image)
    cv.fillPoly(mask, polygons, 255)
    masked_image = cv.bitwise_and(image, mask)
    return masked_image


In [36]:
img = cv.imread("fullNewTrack.jpg")
cv.imshow('img',img)
src = cv.GaussianBlur(img, (3, 3), 0)
scale = 1
delta = 0
ddepth = cv.CV_16S


gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)



grad_x = cv.Sobel(gray, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)

# Gradient-Y
# grad_y = cv.Scharr(gray,ddepth,0,1)
grad_y = cv.Sobel(gray, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)



abs_grad_x = cv.convertScaleAbs(grad_x)
abs_grad_y = cv.convertScaleAbs(grad_y)



grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

_, thresh1 = cv.threshold(gray,200,255,cv.THRESH_BINARY)

edges = cv.Canny(thresh1,100,200)
raw_lines = cv.HoughLinesP(edges,rho = 1,theta = 1*np.pi/180,threshold = 25,minLineLength = 50,maxLineGap = 15)
if raw_lines is not None:
    for line in raw_lines:
        x1, y1, x2, y2 = line[0]
        # print(type(line[0][0]))
        cv.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv.imshow("grad",grad)
cv.imshow("thresh1",thresh1)
cv.imshow("edges",edges)
cv.imshow("img",img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
def trim_lines(lines,width):
    for reference_line in lines:
        for cut_line in lines:
            x1r, y1r, x2r, y2r = reference_line[0]
            x1c, y1c, x2c, y2c = cut_line[0]
            print(reference_line[0])
            print(cut_line[0])
            print("+++")
            if x1r != x1c and y1r != y1c and x2r != x2c and y2r != y2c:
                if (y1r < y1c < y2r or y1r > y1c > y2r) or (y1r < y2c < y2r or y1r > y2c > y2r):
                    if x1r < width//2 and x2r < width//2:
                        if x1r > x1c+5 and x2r > x2c+5 and x1r > x2c+5 and x1r > x2c+5:
                            cut_line[0] = [0,0,0,0]
                            print(reference_line[0])
                            print(cut_line[0])
                            print("----")
                    if x1r > width//2 and x2r > width//2:
                        if x1r +5 < x1c and x2r +5 < x2c and x1r +5 < x2c and x1r +5 < x2c:
                            cut_line[0] = [0,0,0,0]
                            cut_line[0] = [0,0,0,0]
                            print(reference_line[0])
                            print(cut_line[0])
                            print("----")
    return lines

# lines = [[8,6,11,5],[10,7,7,10]]
# print(trim_lines(lines,500))

In [ ]:
def calculate_threshold(left_cords, right_cords, percen):
    """
    @param left_cords array of left lines Coordinates
    @param right_cords array of right lines Coordinates
    @param percen percentile value
    """
    cosine_dist = []
    for left_line in left_cords:
        for right_line in right_cords :
             cosine_dist.append(cosine(left_line, right_line))
    cosine_dist = [item for item in cosine_dist if not math.isnan(item)]
    threshold = np.percentile(cosine_dist, percen)
    return threshold

# Filter the Noise lines from all the lines.
def filter_noise_lines(left_cords, right_cords,threshold):
    """
    @param left_cords array of left lines Coordinates
    @param right_cords array of right lines Coordinates
    @param threshold constraint value of the cosine distance
    """
    final_left_lines = []
    final_right_lines = []

    for left in left_cords: 
        for right in right_cords:
            dist = cosine(left, right)

            if dist > threshold:
                if list(left) not in final_left_lines:
                    final_left_lines.append(list(left))
                if list(right) not in final_right_lines:
                    final_right_lines.append(list(right))
    return [final_left_lines, final_right_lines]
def average_slope_intercept(image, lines):
    """
    0. Iterate over all the lines.q
    1. polyfit - calculate slope and intercept of the lines. (slope, intercept) 
    2. slope < 0 :- Left lane, 
       slope > 0 :- Right lane.
    3. Divide the lines into seperate lists.q
    """
    left_lines, right_lines= [], []
    left_cords, right_cords = [], []

    for line in lines:
        x1, y1, x2, y2 = line.reshape(4) # From Hough Lines
        slope, intercept = np.polyfit((x1, x2), (y1, y2), 1)

        if slope < -0.1:
            left_lines.append([slope, intercept])
            left_cords.append(calculate_coordinates(image, (slope, intercept)))

        elif (slope > 0.2):   
            right_lines.append([slope, intercept])
            right_cords.append(calculate_coordinates(image, (slope, intercept)))
    if len(left_cords)==0 or len(right_cords)==0:
        if len(left_cords)==0:
            lines = right_cords
        else:
            lines = left_cords

        if len(lines)<=2:
            return np.array(lines)
        
        threshold = calculate_threshold(lines, lines, 80)
        final_left_lines, final_right_lines = filter_noise_lines(lines, lines, threshold)

    else:
        threshold = calculate_threshold(left_cords, right_cords, 85)
        final_left_lines, final_right_lines = filter_noise_lines(left_cords, right_cords, threshold)


    left_line = np.average(np.array(final_left_lines), axis=0)
    right_line = np.average(np.array(final_right_lines), axis=0)
    return np.array([left_line, right_line]) 

In [ ]:
img = cv.imread("fullNewTrack.jpg")
# cv.imshow('img',img)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
low_yellow = (29,0,144)
high_yellow = (255,40,255)
lane_image = np.copy(img)
img_hsv = cv.cvtColor(lane_image, cv.COLOR_BGR2HSV)
mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,1))
mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_OPEN, kernel)
edges = cv.Canny(mask_yellow_hsv,100,200)
raw_lines = cv.HoughLinesP(edges,rho = 1,theta = 1*np.pi/180,threshold = 25,minLineLength = 50,maxLineGap = 15)
lines = raw_lines
originalImg = img.copy()
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # print(type(line[0][0]))
        cv.line(originalImg, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # cv.imshow(str(x1)+" "+str(y1)+" "+str(x2)+" "+str(y2),originalImg)
lines = trim_lines(raw_lines,width=img.shape[1])
# lines = raw_lines
cv.imshow('originalImg',originalImg)
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # print(type(line[0][0]))
        cv.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
# print(raw_lines)
# averaged_lines = average_slope_intercept(lane_image, raw_lines,3)
# line_image = display_lines(lane_image, averaged_lines)

# combo_image = cv.addWeighted(lane_image, 0.1, line_image, 1, 1)

# cv.imshow('result',combo_image)
# img1_bg = cv.bitwise_and(img_hsv,img_hsv,mask = mask_yellow_hsv)
# mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_CLOSE, kernel)
# cv.imshow('1', img1_bg)
# cv.imshow('111', mask_yellow_hsv)
# mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
# kernel = cv.getStructuringElement(cv.MORPH_CROSS,(3,1))
# mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_OPEN, kernel)
# cv.imshow('1111', mask_yellow_hsv)
# mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
# kernel = cv.getStructuringElement(cv.MORPH_CROSS,(1,3))
# mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_OPEN, kernel)
# cv.imshow('mask_yellow_hsv', mask_yellow_hsv)
# cv.imshow('edges', edges)
cv.imshow('img+lines',img)
# low_yellow = (0, 0, 0)
# high_yellow = (14, 255, 255)
# img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
# mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
# # kernel = np.ones((6,6),np.uint8)
# mask_yellow_hsv = cv.bitwise_not(mask_yellow_hsv)
# img1_bg = cv.bitwise_and(img_hsv,img_hsv,mask = mask_yellow_hsv)
# # mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_CLOSE, kernel)
# cv.imshow('2', img1_bg)
# cv.imshow('22', mask_yellow_hsv)
cv.waitKey(0)
cv.destroyAllWindows()

[391 184 518 342]
[391 184 518 342]
+++
[391 184 518 342]
[409 187 637 449]
+++
[391 184 518 342]
[0 0 0 0]
----
[391 184 518 342]
[397 172 546 344]
+++
[391 184 518 342]
[ 15 384 111 195]
+++
[391 184 518 342]
[312  86 448 254]
+++
[391 184 518 342]
[516 341 610 457]
+++
[391 184 518 342]
[0 0 0 0]
----
[391 184 518 342]
[ 45 377 154 131]
+++
[391 184 518 342]
[280  44 391 163]
+++
[391 184 518 342]
[323  89 382 152]
+++
[391 184 518 342]
[105 204 139 139]
+++
[391 184 518 342]
[147 121 180  40]
+++
[391 184 518 342]
[498 318 621 470]
+++
[391 184 518 342]
[0 0 0 0]
----
[391 184 518 342]
[  0 416  46 325]
+++
[391 184 518 342]
[264  34 343 122]
+++
[391 184 518 342]
[  4 472  57 352]
+++
[391 184 518 342]
[536 334 622 433]
+++
[391 184 518 342]
[0 0 0 0]
----
[391 184 518 342]
[149 144 174  73]
+++
[391 184 518 342]
[127  26 180  36]
+++
[391 184 518 342]
[ 36 398 118 214]
+++
[0 0 0 0]
[391 184 518 342]
+++
[0 0 0 0]
[0 0 0 0]
+++
[0 0 0 0]
[397 172 546 344]
+++
[0 0 0 0]
[ 15 384 1

In [25]:
import time

cap = cv.VideoCapture('fullNewTrack.mp4')
while(cap.isOpened()):
    time.sleep(1/4)
    _, frame = cap.read()
    # gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    # low_yellow = (29,0,144)
    # high_yellow = (255,40,255)
    # lane_image = np.copy(frame)
    # img_hsv = cv.cvtColor(lane_image, cv.COLOR_BGR2HSV)
    # mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
    # kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,1))
    # mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_OPEN, kernel)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 

    # applying Otsu thresholding 
    # as an extra flag in binary 
    # thresholding
    blur = cv.GaussianBlur(gray,(5,5),0)
    _, thresh1 = cv.threshold(blur,215,255,cv.THRESH_BINARY)
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,1))
    thresh1 = cv.morphologyEx(thresh1, cv.MORPH_OPEN, kernel)
    edges = cv.Canny(thresh1,100,200)
    # raw_lines = cv.HoughLinesP(edges,rho = 1,theta = 1*np.pi/180,threshold = 75,minLineLength = 50,maxLineGap = 25)
    raw_lines = cv.HoughLines(edges,rho = 1,theta = 1*np.pi/180,threshold = 75)
    print(raw_lines)
    # if raw_lines is not None:
    #     for line in raw_lines:
    #         x1, y1, x2, y2 = line[0]
    #         cv.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # print(raw_lines)
    # averaged_lines = average_slope_intercept(lane_image, raw_lines,3)
    # line_image = display_lines(lane_image, averaged_lines)

    # combo_image = cv.addWeighted(lane_image, 0.1, line_image, 1, 1)

    # cv.imshow('result',combo_image)
    # img1_bg = cv.bitwise_and(img_hsv,img_hsv,mask = mask_yellow_hsv)
    # mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_CLOSE, kernel)
    # cv.imshow('1', img1_bg)
    # cv.imshow('111', mask_yellow_hsv)
    # mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
    # kernel = cv.getStructuringElement(cv.MORPH_CROSS,(3,1))
    # mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_OPEN, kernel)
    # cv.imshow('1111', mask_yellow_hsv)
    # mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)
    # kernel = cv.getStructuringElement(cv.MORPH_CROSS,(1,3)) 
    # mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_OPEN, kernel)
    # cv.imshow('mask_yellow_hsv', mask_yellow_hsv)
    cv.imshow('edges', edges)
    cv.imshow('thresh1',thresh1)
    cv.imshow('img+lines',frame)
    # cv.imshow('result',frame)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

[[[-74.          2.3561945]]

 [[167.          1.7976891]]

 [[162.          1.7976891]]

 [[-57.          2.3561945]]

 [[-63.          2.338741 ]]

 [[-85.          2.3736477]]

 [[165.          1.8151424]]

 [[-67.          2.3736477]]

 [[123.          1.9722221]]

 [[-77.          2.3911011]]

 [[159.          1.8151424]]

 [[ 60.          2.1293018]]

 [[117.          1.9896753]]]
[[[-74.          2.3561945]]

 [[167.          1.7976891]]

 [[162.          1.7976891]]

 [[-63.          2.338741 ]]

 [[-57.          2.3561945]]

 [[164.          1.8151424]]

 [[-85.          2.3736477]]

 [[-67.          2.3736477]]

 [[159.          1.8151424]]

 [[123.          1.9722221]]

 [[ 60.          2.1293018]]

 [[-77.          2.3911011]]

 [[117.          1.9896753]]]
[[[-74.          2.3561945]]

 [[167.          1.7976891]]

 [[162.          1.7976891]]

 [[-63.          2.338741 ]]

 [[-57.          2.3561945]]

 [[165.          1.8151424]]

 [[-85.          2.3736477]]

 [[-67.   